# Complete code to replicate our results
This code isn't really user-friendly, but it is somewhat straight-forward to use, since you should only change some variables to test a lot of possibilities ;)

The pytorch part was based on the official pytorch tutorial for neural style transfer (https://pytorch.org/tutorials/advanced/neural_style_tutorial.html)

I ran the code using the following 

First, we need to load all libraries and define some important functions.

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import copy

from time import perf_counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from tqdm import tqdm

from os import walk
from pathlib import Path

def format_time(sec):
    """function just to format seconds to a nice and readable string"""
    h = sec // 3600
    sec -= h*3600
    m = sec // 60
    sec -= m*60
    
    return f'{int(h):02d}h {int(m):02d}m {sec:.2f}'

# runs the code using cuda if a nvidia gpu is avalable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# reconvert from tensor to a PIL image
unloader = transforms.ToPILImage()  # reconvert into PIL image

def tensor2pil(tensor):
    """function to convert tensor to PIL image"""
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    return image

class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product
    return G.div(a * b * c * d)

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

class TotalVariationLoss(nn.Module):
    def __init__(self):
        super(TotalVariationLoss, self).__init__()

    def forward_bkp(self, input):
        bsize, chan, height, width = input.size()
        dy = torch.abs(input[:,:,1:,:] - input[:,:,:-1,:])
        dyhat = torch.abs(input[:,:,1:,:] - input[:,:,:-1,:])
        error = torch.norm(dy - dyhat, 1)
        self.loss = error / height
        return input
    
    def forward(self, input):
        bsize, chan, height, width = input.size()
        a = torch.pow(input[:,:,:-1,:-1] - input[:,:,1:,:-1], 2.)
        b = torch.pow(input[:,:,:-1,:-1] - input[:,:,:-1,1:], 2.)
        self.loss = torch.sum(torch.pow(a+b, 1.25))
        return input

# this loads the pre trained VGG19 model, which was trained using ImageNet dataset
cnn = models.vgg19(pretrained=True).features.to(device).eval()


cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

PS: VGG19 has in total 19 layers and we were supposed to use them all, but since I saw that the last ten layers didn't contribute significantly to the output image, I cut them off and we can only set the weight for the 10 first layers.  
We can see below which layers I chose and naturally you can choose all of them, another configuration, only one, ...  

Note that I chose the conv_5 layer as 

In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_5']
style_layers_default = ['conv_1', 
                        'conv_2', 
                        'conv_3', 
                        'conv_4', 
                        'conv_5',
                        'conv_6', 
                        'conv_7', 
                        'conv_8', 
                        'conv_9', 
                        'conv_10']

"""                     'conv_11', 
                        'conv_12', 
                        'conv_13', 
                        'conv_14', 
                        'conv_15',
                        'conv_16'"""

def get_style_model_and_losses(cnn, 
                               normalization_mean, 
                               normalization_std,
                               style_img, 
                               content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)
    
    #target = model(content_img).detach()
    total_variation_loss = TotalVariationLoss()
    model.add_module('tv_loss', total_variation_loss)
    
    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or \
        isinstance(model[i], StyleLoss) or \
        isinstance(model[i], TotalVariationLoss):
            break
    
    model = model[:(i + 1)]

    return model, style_losses, content_losses, total_variation_loss

def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

def run_style_transfer(cnn, 
                       normalization_mean, 
                       normalization_std,
                       content_img, 
                       style_img, 
                       input_img, 
                       num_steps=300,
                       style_weight=1e5,
                       style_sub_weights=[1]*16, 
                       content_weight=1,
                       tv_weight=1,
                       verbose=3,
                       content_layers=content_layers_default):
    """Run the style transfer."""
    
    if verbose==3:
        print('Building the style transfer model..')
    
    model, style_losses, content_losses, total_variation_loss = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img, content_layers=content_layers)
    optimizer = get_input_optimizer(input_img)

    if verbose==3:
        print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)
    
            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0
    
            for n,sl in enumerate(style_losses):
                style_score += style_sub_weights[n]*sl.loss
            for cl in content_losses:
                content_score += cl.loss
            total_variation_score = tv_weight*total_variation_loss.loss
    
            style_score *= style_weight
            content_score *= content_weight
            
            loss = 0
            loss += style_score if style_weight != 0 else 0
            loss += content_score if content_weight != 0 else 0
            loss += total_variation_score if tv_weight != 0 else 0
            loss.backward()
            
            run[0] += 1
            if run[0] % 100 == 0:
                if verbose>=1:
                    print("run {} - total loss {:.4f}".format(run[0], loss.item()))
                if verbose>=2:
                    print('Style Loss : {:4f} Content Loss: {:4f} Total Variation Loss: {:4f}'.format(
                    style_score.item(), content_score.item(), total_variation_score.item()))
                if verbose>=1:
                    print()
    
            return style_score + content_score + total_variation_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

Until here, we declared only functions and some variables, but nothing really interesting...

But now, we can run experiments!
Above, we have some vars to declare.

- style_weight: This is the weight we will put to the style loss function during the optimization step
- content_weight: Almost the same as style_weight, but for the content loss function.
- tv_weight: Almost the same agein, but for the total variation loss function, which I didn't use on my paper
- num_iterations: number of iterations of the optimizing step
- verbose: 0 for no messages, 1-2 to some level of messages and 3 for see all the details while the network works ;)

Another group of variables are a little bit complexer than the others, but not so much.  
The style_sub_weights are the weights for the 10 first layers from the VGG 19.

In [ ]:
style_weight = 1e4
content_weight = 1
tv_weight = 0
num_iterations = 200
verbose = 0

style_sub_weights = [1., 1.,
                     1., 1.,
                     1., 1., 1., 1.,
                     1., 1.]

# used only to save the style transfer output
style_weights_str = '-'.join([f'{i:02.1f}' for i in style_sub_weights])
content_name = 'cx9' #'.'.join(end_img_base.split('/')[-1].split('.')[:-1])

This is the part we set out input data.

My version of the code support "3D" input, in other words, a serie of 2D images forming a 3D block.  
Considering that I can not share the data, I tried to make this part of the code more understandable and easy to change.

The main loading part is intended to load a dataset formed for a path which contains some images inside.  
If you want to change this do load only one image, 

In [ ]:
# Do the ST in the Z axis
img_ref_filename  = '' # including extension
output_path = ''

# desired size of the output image
# use small size if no gpu
imsize = (<height>, <width>) if torch.cuda.is_available() else (<reduced height>, <reduced width>)

# List of paths to load a dataset
# it can be set to load files from as many paths as you wish
# the only must-do is name the images in alphabetical order, like img_0001, img_0002, ...
images_orig_path = [Path('<path #1>'),
                    Path('<path #2>')]

# this will load all images inside the paths defined above
n = 0
orig_sample = np.zeros((553,464,512), dtype=np.uint8)
for p in images_orig_path:
    files_orig = next(walk(p))[2]
    
    # remove some annoying file starting with a ".".
    # Well, you can filter as many files as you want here!
    files_orig = list(filter(lambda x: x[0] != '.', files_orig))
    
    files_orig.sort()

    for file in files_orig:
        img = Image.open(p / file)
        img = img.resize((orig_sample.shape[2], orig_sample.shape[1]))
        orig_sample[n] = np.array(img)
        n += 1
        
# Filter all values near 0 to reduce some random noise inside pores and outside the sample - if there is any exterior part
orig_sample[orig_sample < 15] = 0

In [ ]:
loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor

def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

style_img = image_loader(img_ref_filename)

tempo_antes_total = perf_counter()

pbar = tqdm(total=orig_sample.shape[0])
output_image = np.zeros_like(orig_sample)

for n in range(orig_sample.shape[0]):
    tempo_antes_parcial = perf_counter()
    
    content_img = loader(Image.fromarray(orig_sample[n]))
    content_img = content_img.unsqueeze(0)
    content_img = content_img.to(device, torch.float)
    
    base_img = content_img.clone()
    job_string = f'{content_name}-sl-{n:03d}-stw-{style_weight:.1e}-stws-{style_weights_str}-cw-{content_weight:.1f}-tvw-{tv_weight:.1f}-it-{num_iterations}.bmp'
    if verbose >= 1:
        pbar.set_description(f'Running {job_string}')

    output = run_style_transfer(cnn, 
                                cnn_normalization_mean, 
                                cnn_normalization_std,
                                content_img.clone(), 
                                style_img.clone(), 
                                base_img.clone(), 
                                num_steps=num_iterations, 
                                style_weight=style_weight,
                                style_sub_weights=style_sub_weights,
                                content_weight=content_weight,
                                tv_weight=tv_weight,
                                verbose=verbose)

    img_output = tensor2pil(output)
    output_image[n] = np.array(img_output)
    img_output.save(f'{output_path}/{job_string}')
    pbar.update(1)

print(f'Total ellapsed time: {format_time(perf_counter() - tempo_antes_total)}')

Here you can explore a little of the output from the style transfer method.  
As the main subject of my paper is porosity, you can choose a threshold and calculate the porosity after and before ST.

In [ ]:
# Analyze porosity
threshold = 100
image_number = 0

img_original_np = orig_sample[image_number]
img_output_np = output_image[image_number]

poro_orig = np.where(img_original_np <= threshold)[0].size/img_original_np.size
poro_output = np.where(img_output_np <= threshold)[0].size/img_output_np.size

print('Porosity:')
print(f' - Original: {100*poro_orig:.2f}%')
print(f' -   Styled: {100*poro_output:.2f}%')

segm_original, segm_st = img_original_np.copy(), img_output_np.copy()

segm_original[segm_original <= threshold] = 0
segm_original[segm_original > threshold] = 1

segm_st[segm_st <= threshold] = 0
segm_st[segm_st > threshold] = 1

plt.figure(0, figsize=(15,15))
plt.subplot(221)
plt.title('Original image')
plt.imshow(img_original_np, cmap='gray')

plt.subplot(222)
plt.title('Stylized image')
plt.imshow(img_output_np, cmap='gray')

plt.subplot(223)
plt.title('Original - Segmented')
plt.imshow(segm_original, cmap='gray')

plt.subplot(224)
plt.title('Stylized - Segmented')
plt.imshow(segm_st, cmap='gray')

plt.tight_layout()
plt.show()